In [ ]:
# questions and personal notes
#problems with reading this as $600 value for W/D - linear regression
#('w/d hookups', -358.55112862600521), ('w/d in unit', 323.94738691104828),

#Still need to remove (dummmy variable values that have only a few instances - e.g. only one person from city)

In [1]:
# imports
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from math import exp

# allow plots to appear directly in the notebook
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv(r'C:\Users\alsherman\Desktop\GitHub\DataScience_GeneralAssembly\Data\Cleaned_Data_May_18.csv')

In [3]:
data.describe()

,Unnamed: 0,ID,latitude,location_data_accuracy,longitude,average_image_size,bedroom,cat,dog,image_number,...,square_footage,grocery_list,gym_list,movie_theatre_list,train_station_list,airport_list,subway_station_list,Barnes_and_Nobles_list,Deloitte_list,Starbucks_list
count,4105.000000,4.105000e+03,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105,4105,4105.000000,...,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105,4105.000000,4105.000000,4105.000000,4105.000000
mean,2052.000000,4.989281e+09,30.350735,1.463337,-60.146224,214092.570037,1.979537,0,0,9.176614,...,1172.067027,9.770280,6.551766,1.023630,2.093788,0,1.403410,0.162241,0.589038,3.264312
std,1185.155756,1.327768e+07,16.201643,4.825632,32.129647,109417.923016,1.224375,0,0,7.621897,...,536.728287,8.218068,7.037377,1.659833,3.864635,0,2.921152,0.490640,1.284344,4.952115
min,0.000000,4.959352e+09,-85.050342,0.000000,-96.045800,0.000000,0.000000,0,0,0.000000,...,20.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
25%,1026.000000,4.976029e+09,38.719200,0.000000,-77.120848,270000.000000,1.000000,0,0,3.000000,...,820.000000,1.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,0.000000
50%,2052.000000,4.991315e+09,38.860195,0.000000,-77.051425,270000.000000,2.000000,0,0,8.000000,...,1172.067027,9.000000,4.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,1.000000
75%,3078.000000,5.001991e+09,38.936759,0.000000,-76.843400,270000.000000,3.000000,0,0,14.000000,...,1295.000000,19.000000,12.000000,2.000000,3.000000,0,2.000000,0.000000,0.000000,4.000000
max,4104.000000,5.005417e+09,41.333300,22.000000,179.992769,270000.000000,6.000000,0,0,24.000000,...,10000.000000,20.000000,20.000000,9.000000,20.000000,0,18.000000,3.000000,6.000000,20.000000


In [4]:
# Transform categorical fields to numeric for scikit-learn 

data.country = np.where(data.country == 'US',1,0)
data.availability = np.where(data.availability == 'available now',1,0)

In [5]:
# Create dummy variables

data = pd.concat([data, pd.get_dummies(data.housing_type)], axis=1)
data = pd.concat([data, pd.get_dummies(data.laundry)], axis=1)
data = pd.concat([data, pd.get_dummies(data.parking)], axis=1)
data = pd.concat([data, pd.get_dummies(data.city)], axis=1)
data = pd.concat([data, pd.get_dummies(data.state)], axis=1)

data[0:1]

,Unnamed: 0,ID,city,country,latitude,location_data_accuracy,longitude,state,availability,average_image_size,...,md,park,potomac,rainier,ridge,spring,springs,va,village,washington
0,0,4959351766,NaN,0,0,0,0,NaN,1,270000,...,0,0,0,0,0,0,0,0,0,0


In [6]:
feature_cols = ['latitude',	 'location_data_accuracy',	 'longitude',	 'average_image_size',	 'bedroom',	 'cat',	 'dog',	 'image_number',	 'smoking',	 'square_footage',	 'grocery_list',	 'gym_list',	 'movie_theatre_list',	 'train_station_list',	 'airport_list',	 'subway_station_list',	 'Barnes_and_Nobles_list',	 'Deloitte_list',	 'Starbucks_list',	 'No_Housing_Data',	 'apartment',	 'condo',	 'cottage/cabin',	 'duplex',	 'flat',	 'house',	 'in-law',		
 'townhouse',	 'No_Laundry_Data',	 'laundry in bldg',	 'laundry on site',	 'w/d hookups',	 'w/d in unit',	 'No_Parking_Data',	 'attached garage',	 'carport',	 'detached garage',	 'off-street parking',	 'street parking',	 'aldie',	 'alexandria',	 'annandale',	 'arlington',	 'ashburn',	 'baltimore',	 'beltsville',	 'bethesda',	 'bladensburg',	 'bowie',	 'broadlands',	 'capitol',	 'centreville',	 'chantilly',	 'chevy',	 'clinton',	 'college',
 'columbia',	 'crofton',	 'dale',	 'damascus',	 'dumfries',	 'elkridge',	 'fairfax',	 'falls',	 'forestville',	 'frederick',	 'fredericksburg',	 'gaithersburg',	 'germantown',	 'greenbelt',	 'hamilton',	 'harpers',	 'herndon',	 'hyattsville',	 'kensington',	 'landover',	 'largo',	 'laurel',	 'laytonsville',	 'leesburg',	 'lorton',	 'manassas',	 'mclean',	 'montgomery',	 'n',
 'north',	 'northwest',	 'oak',	 'oakton',	 'oxon',	 'potomac',	 'reston',	 'riverdale',	 'rockville',	 'rosslyn',	 'silver',	 'springfield',	 'stafford',	 'sterling',	 'stone',	 'suitland',	 'takoma',	 'triangle',	 'upper',	 'vienna',	 'waldorf',	 'washingtondc',	 'wheaton',	 'woodbridge',	 'bethesda',	 'chase',	 'church',	 'college',	 'dc',
 'falls',	 'ferry',	 'heights',	 'hill',	 'marlboro',	 'park',	 'potomac',	 'ridge',	 'spring',	 'village',	 'washington']

## Linear Regression

In [10]:
### SCIKIT-LEARN ###

# create X and y
X = data[feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

# instantiate and fit
lm2 = LinearRegression()
lm2.fit(X_train, y_train)

y_pred = lm2.predict(X_test) # Predict
# print the intercept
#print lm2.intercept_

# pair the feature names with the coefficients
#print zip(feature_cols, lm2.coef_)

# Access accuracy
y_pred = [int(y) for y in y_pred]
y_test = [int(y) for y in y_test]

print metrics.accuracy_score(y_test, y_pred)

0.000973709834469


## Logistic Regression

In [8]:
# create X and y

#feature_cols = []

X = data[feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

# Fit model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test) # Predict

# Access accuracy
print metrics.accuracy_score(y_test, y_pred)

0.246348588121


In [89]:
# Create a Confusion Matrix

con_mat = metrics.confusion_matrix(y_test, y_pred)
print con_mat

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
